#  STILL I AM WORKING ON IT.
#not testesd yet, just started to translate
### Ingestion to COSMOSDB 


#### Set environment variables

In [ ]:
! pip install numpy
! pip install openai
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos
! pip install tenacity

In [146]:
import json
import datetime
import time

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential
from azure.cosmos import exceptions, CosmosClient, PartitionKey
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.models import Vector
from azure.search.documents.indexes.models import (
    IndexingSchedule,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    SemanticConfiguration,
    SimpleField,
    PrioritizedFields,
    SemanticField,
    SemanticSettings,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    SearchIndexerDataSourceConnection
)

import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

import os
from dotenv import load_dotenv


In [ ]:

load_dotenv()

pg_host  = os.getenv("COSMOSDB_HOST")
if pg_host is None or pg_host == "":
    print("COSMOSDB_HOST environment variable not set.")
    exit()

pg_user  = os.getenv("COSMOSDB_USERNAME")
if pg_user is None or pg_user == "":
    print("COSMOSDB_USERNAME environment variable not set.")
    exit()

pg_password  = os.getenv("COSMOSDB_PASSWORD")
if pg_password is None or pg_password == "":
    print("COSMOSDB_PASSWORD environment variable not set.")
    exit()

db_name  = os.getenv("COSMOSDB_DATABASE")
if db_name is None or db_name == "":
    print("COSMOSDB_DATABASE environment variable not set.")
    exit()

text_table_name = 'text_sample'
doc_table_name = 'doc_sample'
image_table_name = 'image_sample'

aoai_key  = os.getenv("AZURE_OPENAI_KEY")



### Cosmos containers


In [ ]:
from azure.cosmos import CosmosClient

# Your Cosmos DB connection parameters
cosmosdb_endpoint = "your_cosmosdb_endpoint"
cosmosdb_key = "your_cosmosdb_key"
database_name= "your_database_name"
cosmosdb_container_name = "your_container_name"

#### Opening the connections.


In [117]:
from azure.cosmos import CosmosClient

# Initialize the Cosmos DB client
client = CosmosClient(cosmos_db_api_endpoint, credential=cosmos_db_api_key)

# Create or get a reference to the database
database = client.create_database_if_not_exists(id=database_name)

print(f"Database {database_name} created or retrieved.")


Database Vector_DB created or retrieved.


####  Initialize the Cosmos DB client - Creating containers


In [81]:


index_client = SearchIndexClient(cosmos_db_api_endpoint, credential=cosmos_db_api_key)

# Define index fields for text_sample
text_index_name = "text_sample_index_vector_search"
text_fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="content", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True, searchable=True, retrievable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
]

# Define index fields for doc_sample
doc_index_name = "doc_sample_index"
doc_fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    # Add other fields relevant to doc_sample
]

# Define index fields for image_sample
image_index_name = "image_sample_index"
image_fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    # Add other fields relevant to image_sample
]

# Your existing vector search configuration
vector_search_config = VectorSearchAlgorithmConfiguration(
    name="my-vector-config",
    kind="hnsw",
    hnsw_parameters={
        "m": 4,
        "efConstruction": 400,
        "efSearch": 1000,
        "metric": "cosine"
    }
)

# Create indexes
text_index = SearchIndex(name=text_index_name, fields=text_fields, vector_search=VectorSearch(algorithm_configurations=[vector_search_config]))
doc_index = SearchIndex(name=doc_index_name, fields=doc_fields)
image_index = SearchIndex(name=image_index_name, fields=image_fields)

# Create or update indexes
index_client.create_or_update_index(text_index)
index_client.create_or_update_index(doc_index)
index_client.create_or_update_index(image_index)

print(f'Indexes created or updated: {text_index_name}, {doc_index_name}, {image_index_name}')


Container created: text_sample
Container created: doc_sample
Container created: image_sample


#### Ingest text sample with embeddings

##### Creating function for ingestion of data

In [98]:

## this adaptation, the script uses the Cosmos DB Python SDK to create items in the Cosmos DB container. 
# The to_dict(orient='records') method is used to convert the Pandas DataFrame to a list of dictionaries, where each dictionary represents a record
# Function to insert data into Cosmos DB
def insert_data(container, records):
    try:
        for record in records:
            # Convert the 'id' attribute to a string
            record['id'] = str(record['id'])
            container.create_item(body=record)
        print(f"Document {container} inserted successfully")

    except exceptions.CosmosResourceExistsError as e:
            # Handle conflict error
            print(f"Document {container}  with ID {record['id']} already exists...")
            print(f"Error: {e}")

            # Implement your logic to update the existing document or take appropriate action

    except Exception as e:
            # Handle other exceptions
            print(f"Error: {e}")


   

#####  Text ingestion

In [99]:
import pandas as pd
from azure.cosmos import CosmosClient


cosmosdb_container_name = text_table_name
container = database.get_container_client(cosmosdb_container_name)

# Read data from the JSON file
text_df = pd.read_json('../data/text/product_docs_embeddings.json')
records = text_df.to_dict(orient='records')


# Insert data into text_sample container
insert_data(container, records)




Document <ContainerProxy [dbs/Vector_DB/colls/text_sample]>  with ID 1 already exists...
Error: (Conflict) Entity with the specified id already exists in the system., {"Summary":{"DirectCalls":{"(409, 0)":1},"RegionsContacted":1},"name":"HandleDocumentRequest","id":"888edc8d-30b5-4401-8504-6d56472bbad3","start time":"11:08:20:205","duration in milliseconds":9.0374,"data":{"Client Side Request Stats":{"Id":"AggregatedClientSideRequestStatistics","ContactedReplicas":[{"Count":1,"Uri":"rntbd://cdb-ms-prod-westus1-be58.documents.azure.com:14431/apps/d220da03-9c5d-440d-9210-240d371d4f82/services/fc1a0c30-d2e1-40bb-bf5b-ec33da8cacff/partitions/e6a67f3f-b30f-48bf-ad96-0ff101c9d868/replicas/133452939844203241p/"},{"Count":1,"Uri":"rntbd://cdb-ms-prod-westus1-be58.documents.azure.com:14071/apps/d220da03-9c5d-440d-9210-240d371d4f82/services/fc1a0c30-d2e1-40bb-bf5b-ec33da8cacff/partitions/e6a67f3f-b30f-48bf-ad96-0ff101c9d868/replicas/133452939872172529s/"},{"Count":1,"Uri":"rntbd://cdb-ms-prod-we

#####  Doc ingestion

In [101]:
cosmosdb_container_name = doc_table_name
container = database.get_container_client(cosmosdb_container_name)

# Read data from the JSON file
doc_df = pd.read_json('../data/docs/employee_handbook_embeddings.json')
records = doc_df.to_dict(orient='records')


# Insert data into doc_sample container
insert_data(container, records)




Document <ContainerProxy [dbs/Vector_DB/colls/doc_sample]>  with ID 0 already exists...
Error: (Conflict) Entity with the specified id already exists in the system., {"Summary":{"DirectCalls":{"(409, 0)":1},"RegionsContacted":1},"name":"HandleDocumentRequest","id":"35d0696f-8409-43d1-93b4-49fb998d0fd8","start time":"11:08:39:509","duration in milliseconds":7.0477,"data":{"Client Side Request Stats":{"Id":"AggregatedClientSideRequestStatistics","ContactedReplicas":[{"Count":1,"Uri":"rntbd://cdb-ms-prod-westus1-be58.documents.azure.com:14049/apps/888bedd0-ff61-4752-a8f0-85dd117f6306/services/5a621f36-05f1-4328-8146-618b6ff8de46/partitions/012b6fb2-aa48-4968-9327-eb3b5ed9bf96/replicas/133452936348987734p/"},{"Count":1,"Uri":"rntbd://cdb-ms-prod-westus1-be58.documents.azure.com:14069/apps/888bedd0-ff61-4752-a8f0-85dd117f6306/services/5a621f36-05f1-4328-8146-618b6ff8de46/partitions/012b6fb2-aa48-4968-9327-eb3b5ed9bf96/replicas/133452936376801242s/"},{"Count":1,"Uri":"rntbd://cdb-ms-prod-wes

#####  Image ingestion

In [102]:
cosmosdb_container_name = image_table_name
container = database.get_container_client(cosmosdb_container_name)

# Read data from the JSON file
doc_df = pd.read_json('../data/docs/employee_handbook_embeddings.json')
records = doc_df.to_dict(orient='records')


# Insert data into doc_sample container
insert_data(container, records)




Document <ContainerProxy [dbs/Vector_DB/colls/image_sample]> inserted successfully


#### Checking the data inserted


In [115]:

# Specify the container name
Table_name = "text_sample"
container = database.get_container_client(Table_name)

#Number of rows
top_x_rows = 10

# Define your query with the COUNT function
#query = "SELECT COUNT(1) as total_rows FROM c"

# Execute the query
#query_result = list(container.query_items(query, enable_cross_partition_query=True))

# Get the total count
#total_rows = query_result[0]['total_rows']
#print(f"Total number of rows inserted - quantity test: {total_rows}")


print(f"Quality test  top ( {top_x_rows} )")

# Define your query with the TOP clause
query = f"SELECT TOP {top_x_rows} * FROM c"

# Execute the query
query_result = container.query_items(query, enable_cross_partition_query=True)

# Process the query result
for item in query_result:
    print(item)


Quality test  top ( 10 )
{'id': '1', 'title': 'Azure App Service', 'content': 'Azure App Service is a fully managed platform for building, deploying, and scaling web apps. You can host web apps, mobile app backends, and RESTful APIs. It supports a variety of programming languages and frameworks, such as .NET, Java, Node.js, Python, and PHP. The service offers built-in auto-scaling and load balancing capabilities. It also provides integration with other Azure services, such as Azure DevOps, GitHub, and Bitbucket.', 'category': 'Web', 'title_vector': [-0.0106361033, -0.021644678, 0.0019778875, -0.0145401461, -0.0219757631, 0.0117742075, -0.0265695658, -0.0086151054, 0.0131951142, -0.0253555868, 0.0146367131, -0.0137814106, 0.0055111847, 0.0016166256, -0.0238381159, 0.014595327000000002, 0.014305627900000001, 0.004390324, 0.0039730193, -0.0035350213000000003, -0.022900041200000002, 0.008146069, 0.014622917400000001, 0.0144435801, -0.0011355178, 0.008573720200000001, 0.0030452916, -0.03261

#### Create HSNW Index



In [157]:
from azure.core.credentials import AzureKeyCredential  
from azure.core.exceptions import HttpResponseError

cog_search_cred = AzureKeyCredential(cog_search_key)  # https://learn.microsoft.com/en-gb/azure/search/search-create-service-portal

try:
    index_client = SearchIndexClient(endpoint=cog_search_endpoint, credential=cog_search_cred)


    # Define index fields for text_sample
    text_index_name = "hsnw_sample_index_vector_search"
    text_fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="title", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchableField(name="content", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchableField(name="category", type=SearchFieldDataType.String,
                        filterable=True, searchable=True, retrievable=True),
        SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, dimensions=1536, vector_search_configuration="vector-cosmos-config"),
        SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, dimensions=1536, vector_search_configuration="vector-cosmos-config"),
    ]

    # Define index fields for doc_sample
    doc_index_name = "doc_sample_index"
    doc_fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="chunk_content", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchField(name="chunk_content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, dimensions=1536, vector_search_configuration="vector-cosmos-config"),        
    ]

    # Define index fields for image_sample
    image_index_name = "image_sample_index"
    image_fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="image", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchField(name="image_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, dimensions=1536, vector_search_configuration="vector-cosmos-config"),        
    ]
    

    # Your existing vector search configuration
    ##https://github.com/microsoft/AzureDataRetrievalAugmentedGenerationSamples/blob/main/Python/CosmosDB-NoSQL_CognitiveSearch/CosmosDB_CogSearch_AzureOpenAI_Tutorial.ipynb
    vector_search_config = VectorSearchAlgorithmConfiguration(
        name="vector-cosmos-config",
        kind="hnsw",
        hnsw_parameters={
            "m": 4,
            "efConstruction": 400,
            "efSearch": 1000,
            "metric": "cosine"
        }
    )
    # Create indexes
    text_index = SearchIndex(name=text_index_name, fields=text_fields, vector_search=VectorSearch(algorithm_configurations=[vector_search_config]))
    doc_index = SearchIndex(name=doc_index_name, fields=doc_fields, vector_search=VectorSearch(algorithm_configurations=[vector_search_config]))
    image_index = SearchIndex(name=image_index_name, fields=image_fields, vector_search=VectorSearch(algorithm_configurations=[vector_search_config]))


    # Create or update indexes
    #index_client.create_or_update_index(text_index)
    print(f'Indexes created or updated: {text_index_name}')
    
    index_client.create_or_update_index(doc_index)
    print(f'Indexes created or updated: {doc_index_name}')

    index_client.create_or_update_index(image_index)
    print(f'Indexes created or updated: {image_index_name}')

    
    
except HttpResponseError as e:
    print(f"HTTP Error: {e}")
    print(f"Status Code: {e.status_code}")
    print(f"Error Message: {e.error.message}")





Indexes created or updated: hsnw_sample_index_vector_search
Indexes created or updated: {'additional_properties': {}, 'name': 'doc_sample_index', 'fields': [<azure.search.documents.indexes.models._index.SearchField object at 0x0000028A9B7F3850>, <azure.search.documents.indexes.models._index.SearchField object at 0x0000028A9B7F39D0>, <azure.search.documents.indexes.models._index.SearchField object at 0x0000028A9B7F20D0>], 'vector_search': <azure.search.documents.indexes._generated.models._models_py3.VectorSearch object at 0x0000028A9BC3C4D0>, 'scoring_profiles': None, 'default_scoring_profile': None, 'cors_options': None, 'suggesters': None, 'analyzers': None, 'tokenizers': None, 'token_filters': None, 'char_filters': None, 'normalizers': None, 'encryption_key': None, 'similarity': None, 'semantic_settings': None, 'e_tag': None}
Indexes created or updated: {'additional_properties': {}, 'name': 'image_sample_index', 'fields': [<azure.search.documents.indexes.models._index.SearchField obj